The `codebook.ipynb` notebook has three purposes. First, to parse the taxonomy of open and axial codes in `code_tree.yaml`. Second, to parse all the open codes in PDF printouts of computational notebooks located in the `notebooks/` directory. Third, it calculates code-analysis frequency for each code, the number of unique analyses that contain at least one instance of each open and axial code. It combines all this data in the `codes` data frame and exports this for further analysis in `data/codes.csv`.

In [1]:
import re, yaml
import pandas as pd
from lib.util import getCodes, displayMarkdown

%autosave 0

pd.set_option("display.max_rows", None)  # Don't truncate rows when printing a Pandas DataFrame instance

Autosave disabled


# Parse code tree

Recursively traverse the YAML code tree to transform the data from a tree into tabular form. The node called "root" does not actually exist in the code tree.

In [2]:
with open('code_tree.yaml', 'r') as f:
    code_yaml = yaml.safe_load(f)
    
codes = []

def preTreeWalk(pNode, node, func, lvl=0):
    """ A recursive, pre-order traversal of the code groups YAML structure"""
    leaf = 'sub' not in node.keys()
    func(pNode, node, lvl, leaf)
    if not leaf:
        for child in node['sub']:
            preTreeWalk(node, child, func, lvl + 1)

parseYaml = lambda parent, child, lvl, leaf: codes.append({
    'parent': parent['name'].lower(),
    'name': child['name'].lower(),
    'desc': child['desc'],
    'level': lvl,
    'is_leaf': leaf
})

for grp in code_yaml:
    preTreeWalk({'name': 'root'}, grp, parseYaml)

codes = pd.DataFrame(codes)[['parent', 'name', 'desc', 'level', 'is_leaf']]
codes.head()

,parent,name,desc,level,is_leaf
0,root,actions,Codes that describe actions the journalist has...,0,False
1,actions,import,How raw data is introduced into the programmin...,1,False
2,import,fetch,Data is retrieved from some external sources t...,2,False
3,fetch,pull tables out of pdf,"Using a table extraction tool, such as Tabula,...",3,True
4,fetch,api request,"Make a request to a web API, such as addresses...",3,True


## Display code tree

All open codes, their descriptions, and the corresponding axial codes are stored in the `code_tree.yaml` file. As the master copy for all open and axial codes resides here, the raw text itself can be difficult to read. Thus, it can be helpful to read this tree in Markdown.

In [3]:
minLevel = 5

In [4]:
codeMarkdownTree = [ '{}1. **{}**: {}\n'.format('\t' * c['level'], c['name'].title(), c['desc']) for i, c in codes[codes.level <= minLevel].iterrows() ]

displayMarkdown("""
### Codes\n {}
""".format('\n'.join(codeMarkdownTree)))


### Codes
 1. **Actions**: Codes that describe actions the journalist has taken to wrangle data for further analysis

	1. **Import**: How raw data is introduced into the programming/wrangling environment

		1. **Fetch**: Data is retrieved from some external sources to the programming environment

			1. **Pull Tables Out Of Pdf**: Using a table extraction tool, such as Tabula, to parse tables inside PDF documents.

			1. **Api Request**: Make a request to a web API, such as addresses translation service for lat-long coordinates from Bing.

			1. **Query Database**: Data is imported through a database query

			1. **Scrape Web For Data**: Systematically parsing HTML web pages for relevant data

		1. **Create**: Data is created inside the programming environment

			1. **Construct Table Manually**: Using tables where column names and table values were either copy-and-pasted or entered manually.

			1. **Generate Data Computationally**: Using tables populated programmatically.

			1. **Copy Table Schema**: A table is copied without any values but table column names and type identical, or nearly identical, to another table

			1. **Backfill Missing Data**: Create data observations where there are missing entries.

		1. **Load**: Raw data resides on the local disk and is *loaded* into the environment, includes these file formats: .csv, .xlsx, .fec, .shp, .RData, etc.

	1. **Recalculate**: Expanding the table by calculating new columns based on existing columns without *integrating* other tables.*

		1. **Detrend**: "filter out the secular effect in order to see what is going on specifically with the phenomenon you are investigating," Philip Meyer in *Precision Journalism*.

			1. **Adjust For Inflation**: TK

			1. **Compute Index Number**: TK

			1. **Adjust For Season**: Making seasonal adjustments to the data to detrend for seasonal trends.

		1. **Formulate Performance Metric**: Perform a calculation that is later used to compare different entities or the same entity over time. A recurring theme between many of these notebooks is to compare different entities, such as political parties, by a common, quantitative metric, such as percentage of all newly registered voters.

			1. **Standardize Values**: Measuring deviation from some definition of "normal", e.g. Z-scores

			1. **Figure A Rate**: Convert numbers to a normalized rate to "provide a comparison against some easily recognized baseline" Philip Meyer in *Precision Journalism*

			1. **Calculate Central Tendency**: Measuring what a typical value is in the data, e.g. mean, median.

			1. **Calculate Change Over Time**: Such as the percentage difference over time.

			1. **Calculate Spread**: Calculating the difference between two values or rates

			1. **Domain-Specific Performance Metric**: A domain specific metric, such as the Cube Root Law for legislatures.

			1. **Get Extreme Values**: Calculate the highest or lowest value(s)

	1. **Modify**: *Modifying* the data constitutes make minor changes without *integrating* other tables.

		1. **Encode Table Identification In Row**: When some way of identifying the table is encoded as a separate column in each row. Common identification methods include the name of the corresponding file, an arbitrary table name, or boolean value

		1. **Network-Ify The Data**: When the data is inherently a graph, encode table columns and values to represent this structure tabularly

			1. **Create Edge**: A column with value that define a relationship to another row, which is not necessarily in a different table

			1. **Define Edge Weights**: Columns that define edge weights

		1. **Generate Keys**: Operations that create "key" columns.

			1. **Create Soft Key**: Keys used in matching without a guarentee of uniqueness, such as combinations of names and addresses

			1. **Create A Unique Key**: Journalist create a key that is actually unique in the table

		1. **Rank Data**: Operations that encode semantic meaning about the data with table index.

			1. **Assign Ranks**: When a column of numerical ranks is explicitly assigned to rows in the table.

			1. **Sort Table**: When rank is implicitly assigned by rearranging row position in the table.

		1. **Column Reshaping**: Codes concerning either separating one column into more than one or combining more than one column into one.

			1. **Extract Column Values**: Separating a column containing multiple rows

			1. **Combine Columns**: Combining two columns into one, either through concatenation, addition, etc...

		1. **Create Flag**: Flags are boolean expressed computed upon column values and used in filtering and grouping

		1. **Pad Column Values**: Adding either character prefixes or suffixes consistently to every row within a column

		1. **Scale Values**: Operations that apply some mathematical operation to columns of quantitative data. This code is different from the codes under **Formulate performance metric** because this closer to cleaning.

		1. **Consolidate Values Of A Single Column**: Codes that map a set of entities to a smaller set of entities

			1. **Bin Values**: Consolidating a range of quantitative data into ordinal data

			1. **Combine Entities**: Consolidating a range of categorical values into a smaller set of categorical values

	1. **Clean**: Operations to correct data that might be considered "dirty"

		1. **Trim Fat**: Remove complete sections of data that just are relevant

			1. **Subset Columns**: Removing columns from a table by specifying which ones to remove or keep.

			1. **Winnow Rows**: Simply put, these operations remove table rows.

				1. **Trim By Date Range**: Removing rows that are inside or outside a specific date range.

				1. **Trim By Geographic Area**: Remove rows that are inside or outside the geographic area.

				1. **Trim By Quantitative Threshold**: Remove rows that are above, below, equal to, or not equal to a numeric value.

				1. **Trim By Contains Value**: Remove rows that do or do not contain specific values or types of values.

		1. **Remove Incomplete Data**: Drop row if value(s) are incomplete, usually denoted as NA.

		1. **Deduplicate**: Remove rows that contain two or more of the same observation, with identical values in all, one, or zero columns.

		1. **Fix Values**: Individual values have errors that must be corrected.

			1. **Resolve Entities**: Classic entity resolution: a column of categorical values has different names for the same entity.

			1. **Fix Data Errors Manually**: Instances where individual row-column values are changed manually.

			1. **Fix Mixed Data Types**: Sometimes a column with be mixed with two data type, e.g. integers and strings.

			1. **Remove Value Characters**: When characters inside a value are removed, such as periods, commas, dollar signs, etc

			1. **Replace Na Values**: Raw data may contain incomplete table values (denoted as NA) or empty values (denoted as NULL)

			1. **Strip Whitespace**: Removing extra whitespace characters from entity name

		1. **Format**: Operations that modify the table values appearance or style

			1. **Format Values**: Operations that change value appearence, e.g. change case, specifying date format, rounding floats.

			1. **Format Schema**: Operations that modify anything except table values

				1. **Canonicalize Column Names**: Operations that change column names

				1. **Change Column Data Type**: For example, changing a column of values from strings to integers

	1. **Integrate**: Combining data residing in different tables into one table.

		1. **Union Tables**: TK

		1. **Inner Join Tables**: TK

		1. **Supplement**: Supplementation is characterized by integration operations that essentially add columns to existing data

			1. **Outer Join Tables**: A join that returns rows with no corresponding match in the table being joined two, e.g. left or right joins.

			1. **Full Join Tables**: Combine all rows and all columns of the two tables. a.k.a full outer join

			1. **Concat Parallel Tables**: When columns from multiple, parallel tables are concatenated together to form a new table.

			1. **Use Lookup Table**: Using a table with two columns to map from one value to another.

		1. **Cartesian Product**: TK

		1. **Self Join Table**: Join a table with itself

	1. **Transform**: Operations that transform a table into an aggregated, lower-resolution view of the original table.

		1. **Summarize**: Codes that aggregate and calculate tables to get a more coarse view of the data.

			1. **Rollup**: Rename entity to the name of its parent (for hierarchical data)

			1. **Join Aggregate**: Extend the table (columnwise) with aggregate values, hence the number of rows stays constant but columns increase

			1. **Group By Single Column**: When a table is grouped by a single column.

			1. **Group By Multiple Columns**: When a table is grouped by multiple columns, creating hierarchy.

			1. **Rolling Window Calculation**: Performs rolling-window aggregation

			1. **Create Frequency Table**: Count the frequency of non-quantitative variables within a column

		1. **Reshape**: Operations fundamentally change the table's structure, but do not perform any kind of summarization calculation. *Constructing a pivot table* often involves a *spread-like* operation when defining what values to use as columns in the new table. The difference with *reshaping* is that sometimes the journalist may not summarize the reshaped table.

			1. **Spread Table**: Expand two columns of key value pairs into multiple columns.

			1. **Gather Table**: Collapses table into key value pairs.

			1. **Cross Tabulate**: such as with a pivot table/crosstab

	1. **Display Dataset**: Different ways to check in on the state of the dataset during wrangling.

		1. **Format Table Display**: Operations that adjust the table displace, such as how many decimals to round floats

		1. **Visualize Data**: Employing any kind of data visualization, including a table

		1. **Describe Statistically**: Generates any kind of descriptive statistics of the dataset's central tendency, dispersion and distribution shape

	1. **Check Sanity**: Operations that confirm the effect of a previous wrangling operation.

		1. **Run A Test**: Operations output a clear pass or fail value, often implemented by counting things

			1. **Report Rows With Column Number Discrepancies**: Finds if a row has a different number of columns than the header row

			1. **Test For Equality**: Test if two data structures are exactly the same, e.g. two data frames

			1. **Test Different Computations For Equality**: Test the results of a calculation against different methods/packages. The Upshot did this with variance.

			1. **Validate Data Quality With Domain-Specific Rules**: Such as if the average temperature is higher than the maximum recorded temperature

		1. **Check Results**: Operations that output some visual representation of the table

			1. **Peek At Data**: Display the first *n* rows and all columns of the table

			1. **Inspect Table Schema**: Check the data types of columns

			1. **Display Rows With Missing Values**: E.g. filtering rows with a NA value in a particular column

			1. **Check For Nas**: See if any rows have NA values.

		1. **Count The Data**: Operation that count things in the data set

			1. **Count Number Of Rows**: Printing out the total number of rows in a table

			1. **Count Unique Values**: Report the number of unique values in one or more columns

	1. **Export**: Ways in which journalist export the results of their data wrangling.

1. **Observations**: These codes cover observations from the coder about the wrangling processes, not actions performed by the journalist.

	1. **Data Acquisition**: How the data was acquired by journalists

		1. **Collect Raw Data**: Using first-hand observations or logs as data.

		1. **Use Previously Cleaned Data**: Data that originated from a colleague.

		1. **Use Public Data**: Includes open-source datasets, tables on Wikipedia, etc..

		1. **Use Academic Data**: 

		1. **Use Non-Public, Provided Data**: 

		1. **Use Open Government Data**: Data publically available on open data portals, such as data.gov

		1. **Freedom Of Information Data**: Data that was obtained via FOI/FOIA requests.

		1. **Use Another News Orgs Data**: A dataset previously published by another news organization

		1. **Use Data From Colleague**: A dataset was provided by another journalist.

	1. **Workflow Building**: Codes pertaining to how the wrangling workflow is built.

		1. **Annotate Workflow**: Adding comments or notes in Markdown that explain what the journalists doing.

		1. **Think Computationally**: Codes that demonstrate computational thinking on the part of the journalist.

			1. **Architect A Subroutine**: A set of instructions grouped together to be performed multiple times.

			1. **Architect Repeating Process**: Instances where journalists employed a loop.

		1. **Toggle Step On And Off**: Some wrangling steps were not always run. Toggling off is often accomplished by commenting out code.

	1. **Wrangling Purpose**: Why does this data need to be wrangled?

		1. **Input For Downstream Applications**: Output from wrangling will be input into some other program

			1. **Wrangle Data For Graphics**: Data need to be formatted in order to be visualized in an article, including tables.

			1. **Wrangle Data For Model**: Data is being wrangled in order to create a model, whether the main point of the piece is for prediction or classification

		1. **Remove Erroneous Data**: There are errors in the data that need to be removed

		1. **Creating New Datasets**: 

			1. **Combine Drifting Datasets**: Reconcile difference in periodically published datasets that have superficially changed over time, such as schema differences or entity names, to consolidate more than one dataset.

			1. **Combine Seemingly Disparate Datasets**: When a notebook largely constitutes combining seemingly unrelated datasets.

			1. **Combine Data And Geography**: Pairing data with GIS info.

		1. **Aggregate The Forest From The Trees**: Data of individual observations is aggregated in an attempt to find some meaningful structure or patterns

	1. **Analysis**: Kinds of analysis data journalists need to wrangle data to perform.

		1. **Interpret Statistical/Ml Model**: Analyze features from a model such as linear regression or classification trees

		1. **Compare Different Groups Along A Common Metric**: The end analysis is just comparing different groups by a common metric.

		1. **Identify Extreme Values**: 

		1. **Outlier Detection**: Finding extreme cases or outliers in the data

		1. **Show Trend Over Time**: Analysis consists of showing how values change over time

		1. **Calculate A Statistic**: Calculate a single value for from a dataset, such as number of records.

		1. **Answer A Question**: Analysis consists of using data to answer a specific question

		1. **Examine Relationship**: Analysis consists of examining the relationship between different phenomena

		1. **Explain Variance**: This can be done via PCA

		1. **Identify Clusters Or Lack Of Clusters**: Look for meaningingful groups within the data

		1. **Find Nearest Neighbours In The Network**: (Network analysis) Find the closest neighbours for all points

		1. **Explore Dynamic Network Flow**: (Network analysis) explore the flow between different nodes in the graph, e.g. migration between cities.

	1. **Strategies**: General strategies journalists employ when wrangling data.

		1. **Tables Evolve**: Data and objects are destroyed during the wrangling process.

			1. **Value Replacement**: The output of any column calculation is reassigned to an existing column.

			1. **Temporary Joining Column**: When a key for joining two tables is created and destroyed immediately after the join.

			1. **Refine Table**: Table refinement refers to when a table is subset *in place*, a new object is not created in the environment.

		1. **Data Is Precious**: Data and objects are neverly actually lost in the programming environment.

			1. **Preserve Existing Values**: The output of any column calculation is assigned to a new column

			1. **Create Child Table**: A child table is a subset of the parent table declared as a new object in the environment.

		1. **Set Data Confidence Threshold**: Removes rows where a quantitative value is less than, greater than, or not equal to a numeric value.

		1. **Table Splitting**: Tables may be divided, partitioned, or otherwise split into multiple tables to accomplish a transformation goal.

			1. **Split, Compute, And Merge**: First, the journalist partitions a single data frame into multiple, separate data frames. Then, often identical computations are run on all the data frame. Finally, the multiple data frames are consolidated into one data frame again.

			1. **Split And Compute**: One table is split into two or more and identical computations are applied to each table.

		1. **Tolerate Dirty Data**: Analysis continues despite clear data quality issues.

	1. **Pain Points**: Areas where journalist seem/could be frustrated in the wrangling process.

		1. **Fix Incorrect Calculation**: Calculations in the data are incorrect and the journalist must recalculate them

		1. **Repetitive Code**: Instances where code is repetitively copied and pasted.

		1. **Make An Incorrect Conclusion**: Instances where the journalist has made an incorrect conclusion about the data.

		1. **Post-Merge Clean Up**: Pain points that come from the result of merging two datasets together

			1. **Resort After Merge**: When a sort has to be re-done because a merge ruining the pre-merged order.

			1. **Fill In Na Values After An Outer Join**: As outer joins do not drop non-matching rows, those values have NA

		1. **Encode Redundant Information**: When data that already exists in the table is recoded into the table.

		1. **Post-Aggregation Clean Up**: Pain points that come from the result of grouping a table.

			1. **Data Loss From Aggregation**: When table columns are lost because they were dropped form resulting table due to not being relevant in aggregation.

			1. **Silently Dropping Values After Groupby**: Values other than thsoe being grouped and calculated upon are lost in a group by operation

		1. **Data Too Large For Repo**: Raw data cannot be included in SCM because files are too large



# Parse coded notebooks

For each computational notebook and script used for wrangling data in each analysis, we created PDF printouts with a `.html.pdf`. This extension distinguishes them from possible PDFs checked into the repositories by contributors. All of these printouts fit the glob pattern `notebooks/**/**/*.html.pdf`. We open-coded PDF printouts using the comments feature in [Adobe Acrobat DC](https://acrobat.adobe.com/en/acrobat.html). Open codes are extracted from each PDF using some internals of the open-source [pdfannots CLI](https://github.com/0xabu/pdfannots). See the [main function in pdfannots.py](https://github.com/0xabu/pdfannots/blob/6dd8dd29a93a0f5ec55e4b47f0eb27d8088a11a0/pdfannots.py#L469) for more details. 

The `codeData` data frame links open codes with the notebooks in which they appear. Warning: this cell may take awhile to execute.

In [5]:
%%time
codeData = getCodes()

CPU times: user 1min 9s, sys: 160 ms, total: 1min 9s
Wall time: 1min 9s


## Quality Assurance

This section contains various coding QA measures.

### Matching codes between notebooks and the Code Tree

The cell below ensures that there aren't any codes in the code tree that aren't in the PDF printouts and vice versa. More precisely, it checks that the difference between the set of open codes in `code_tree.yaml` and the set of unique codes that appear in every PDF printout (`notebooks/**/**/*.html.pdf`) is the empty set.

In [6]:
# Parse the code YAML for just the open codes (leaves)
leaves = []
def collectLeaves(node, repo):
    """Recursively traverse dictionary tree and collect only the leave nodes"""
    if 'sub' in node.keys():
        for subnode in node['sub']:
            collectLeaves(subnode, repo)
    else:
        safeCode = node['name'].strip().lower()
        repo.append(safeCode)

for grp in code_yaml:
    collectLeaves(grp, leaves)

# Convert from lists to sets
leaves = set(leaves)
pdf_codes = set(codeData['code'].unique())

# Find any discrepancies
diff = lambda a, b, codes: displayMarkdown('Codes in `{}` but not in `{}`:\n{}\n'.format(a, b, '\n'.join(['* ' + c for c in codes])))

falsePositives = pdf_codes.difference(leaves)
falseNegatives = leaves.difference(pdf_codes)

if not (bool(falsePositives) or bool(falseNegatives)):
    # Both sets are the null set
    displayMarkdown('<p>All codes have been grouped!</p><img src="https://media.giphy.com/media/XreQmk7ETCak0/giphy.gif"> ')
else:
    # Problems
    if len(pdf_codes.difference(leaves)) > 0:
        diff('*.html.pdf', 'code_tree.yaml', pdf_codes.difference(leaves))
    if len(leaves.difference(pdf_codes)) > 0:
        diff('code_tree.yaml', '*.html.pdf', leaves.difference(pdf_codes))

<p>All codes have been grouped!</p><img src="https://media.giphy.com/media/XreQmk7ETCak0/giphy.gif"> 

### Find notebooks with certain codes

If extracted codes and the codes in `code_tree.yaml` don't match, then we can find the corresponding open code by grouping data by code, article, and analysis.

In [7]:
needles = ['split, compute, and merge']

codeData['mark'] = '✔️'

codeData[codeData.code.isin([n.lower() for n in needles ])] \
    [['org', 'analysis', 'notebook', 'code', 'mark']] \
     .drop_duplicates() \
     .set_index(['org', 'analysis', 'notebook', 'code']) \
     .unstack(fill_value='')

mark
code                                                                              split, compute, and merge
org            analysis                           notebook                                                 
la_times       california-ccscore-analysis        analysis.ipynb                                         ✔️
nytimes        gunsales                           analysis.R                                             ✔️
               prison-admissions                  export.rmd                                             ✔️
publicl        employment-discrimination          employment-discrimination.ipynb                        ✔️
star-tribune   201901-hospitalquality             hospital_quality_script.R                              ✔️
stlpublicradio 2018-05-31-crime-and-heat-analysis crimes-and-heat.ipynb                                  ✔️
vox            vox-central-line-infections        build.sh                                               ✔️
wuft           Power_of_Irma                      bailey_code.R                                          ✔️

### Code group gaps

There are some code groups that should be applied to all analyses.

#### Analysis Codes


In [8]:
analysisCodes = codes[codes.parent == 'analysis'].name.unique()
codedAnalyses = set(codeData[codeData.code.isin(analysisCodes)].analysis.unique())
analyses = set(codeData.analysis.unique())
codeDiff = analyses.difference(codedAnalyses)

if len(codeDiff) > 0:
    displayMarkdown("""
    The following analyses do not have analysis codes:

    * {}

    """.format('\n* '.join(codeDiff)))
else:
    displayMarkdown("All analyses have analysis codes")

All analyses have analysis codes

# Calculating code-analysis frequency

Because PDF printouts have only open codes inside, we have to do a little bit of data wrangling to figure out how many analyses 


In [9]:
codeAnalysisTmp = pd.merge(codes[codes.is_leaf].copy()[['parent', 'name']], codeData[['code', 'analysis', 'notebook']],
                        how='left',
                        left_on='name',
                        right_on='code') \
                 .drop(['code'], axis=1) \
                 .drop_duplicates()

codeAnalysis = codeAnalysisTmp[['name', 'analysis', 'notebook']].copy()

while codeAnalysisTmp.parent.nunique() > 0:
    codeAnalysisTmp = codeAnalysisTmp[['parent', 'analysis', 'notebook']] \
        .rename(columns={'parent': 'name'}) \
        .drop_duplicates()

    codeAnalysisTmp = pd.merge(
        codeAnalysisTmp,
        codes[['parent', 'name']],    
        how = 'left',
        on = 'name')

    codeAnalysis = pd.concat([codeAnalysis, codeAnalysisTmp[['name', 'analysis', 'notebook']]]) \
        .drop_duplicates()

codeAnalysis = pd.merge(
    codeAnalysis,
    codes[codes.name != 'root'][['name', 'level', 'is_leaf']], 
    how='left')

In [10]:
codeAnalysisGrp = codeAnalysis.groupby('name')['analysis'].nunique().to_frame('analysis').reset_index()

displayMarkdown('The minimum analysis count for any code should be 1: {}'.format(1 == min(codeAnalysisGrp.analysis)))    

The minimum analysis count for any code should be 1: True

In [11]:
priorSize = codes.shape[0]

codes = pd.merge(codes, codeAnalysisGrp, how='left', on='name')

displayMarkdown(('The data frame `codes` differ by {} rows after the aggregate join'.format(priorSize - codes.shape[0])))
codes.head()

The data frame `codes` differ by 0 rows after the aggregate join

,parent,name,desc,level,is_leaf,analysis
0,root,actions,Codes that describe actions the journalist has...,0,False,50
1,actions,import,How raw data is introduced into the programmin...,1,False,39
2,import,fetch,Data is retrieved from some external sources t...,2,False,6
3,fetch,pull tables out of pdf,"Using a table extraction tool, such as Tabula,...",3,True,1
4,fetch,api request,"Make a request to a web API, such as addresses...",3,True,1


# Export results

We export a couple of CSV files for other notebooks to use.

* `data/codes.csv` contains information on individual axial codes such as their level in the tree and how many analyses in which the code occurs.

* `data/code-analysis-network.csv` contains the occurrence of open and axial codes in individual notebooks.

In [12]:
codes.to_csv('data/codes.csv', index=False)

codeAnalysis.to_csv('data/code-analysis-network.csv', index=False)